In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)


model = Net().to(device)
summary(model, input_size=(1, 28, 28))

#6M params!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (

<ipython-input-2-6de658d552be>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [3]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 10, 5)
        # self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.conv6(F.relu(self.conv5(x)))
        # x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net2().to(device)
summary(model, input_size=(1, 28, 28))

# remove 1024 layer, replace earlier layer by 5x5 kernel
# 1.6M params

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8             [-1, 10, 1, 1]         128,010
Total params: 1,696,010
Trainable params: 1,696,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.44
Params size (MB): 6.47
Estimated Total Size (MB): 7.91

<ipython-input-3-571fe9f096c9>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 256, 3)
        self.conv6 = nn.Conv2d(256, 256, 3)
        self.conv7 = nn.Conv2d(256, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net3().to(device), input_size=(1, 28, 28))

# num ch capped to 256
# 1.5M params

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 256, 5, 5]         590,080
            Conv2d-8            [-1, 256, 3, 3]         590,080
            Conv2d-9             [-1, 10, 1, 1]          23,050
Total params: 1,591,050
Trainable params: 1,591,050
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.41
Params size (MB): 6.07
Estimated Total Size (MB): 7.48
---------------------------------------

<ipython-input-4-406101a928c6>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [5]:
class Net4(nn.Module):
    def __init__(self):
        super(Net4, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(128, 128, 3)
        self.conv6 = nn.Conv2d(128, 128, 3)
        self.conv7 = nn.Conv2d(128, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net4().to(device), input_size=(1, 28, 28))

# num ch capped to 128
#546k params

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 128, 14, 14]         147,584
         MaxPool2d-6            [-1, 128, 7, 7]               0
            Conv2d-7            [-1, 128, 5, 5]         147,584
            Conv2d-8            [-1, 128, 3, 3]         147,584
            Conv2d-9             [-1, 10, 1, 1]          11,530
Total params: 546,954
Trainable params: 546,954
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.13
Params size (MB): 2.09
Estimated Total Size (MB): 3.22
-------------------------------------------

<ipython-input-5-2bb1471b3881>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [6]:
class Net5(nn.Module):
    def __init__(self):
        super(Net5, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(64, 64, 3)
        self.conv6 = nn.Conv2d(64, 64, 3)
        self.conv7 = nn.Conv2d(64, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net5().to(device), input_size=(1, 28, 28))

# num ch capped to 64
#172k params

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          36,928
            Conv2d-5           [-1, 64, 14, 14]          36,928
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7             [-1, 64, 5, 5]          36,928
            Conv2d-8             [-1, 64, 3, 3]          36,928
            Conv2d-9             [-1, 10, 1, 1]           5,770
Total params: 172,298
Trainable params: 172,298
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.90
Params size (MB): 0.66
Estimated Total Size (MB): 1.56
-------------------------------------------

<ipython-input-6-608a7deb819f>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
class Net6(nn.Module):
    def __init__(self):
        super(Net6, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(32, 32, 3)
        self.conv6 = nn.Conv2d(32, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net6().to(device), input_size=(1, 28, 28))

# num ch capped to 32
#50k params
#98.5 in 5 

<ipython-input-7-cc23c86254df>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 32, 28, 28]           9,248
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           9,248
            Conv2d-5           [-1, 32, 14, 14]           9,248
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]           9,248
            Conv2d-8             [-1, 32, 3, 3]           9,248
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 49,450
Trainable params: 49,450
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.55
Params size (MB): 0.19
Estimated Total Size (MB): 0.74
---------------------------------------------

In [8]:
class Net7(nn.Module):
    def __init__(self):
        super(Net7, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(64, 32, 3)
        self.conv6 = nn.Conv2d(32, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net7().to(device), input_size=(1, 28, 28))

# added 64 ch to couple of layer in the middle
#96k params
#98.5 in 5 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 32, 28, 28]           9,248
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
            Conv2d-5           [-1, 64, 14, 14]          36,928
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]          18,464
            Conv2d-8             [-1, 32, 3, 3]           9,248
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 95,594
Trainable params: 95,594
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.65
Params size (MB): 0.36
Estimated Total Size (MB): 1.02
---------------------------------------------

<ipython-input-8-7935126f9900>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [9]:
class Net8(nn.Module):
    def __init__(self):
        super(Net8, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0)   # 26
        self.pool1 = nn.MaxPool2d(2, 2)               # 13
        self.conv2 = nn.Conv2d(16, 32, 3, padding=0)  # 11
        self.pool2 = nn.MaxPool2d(2, 2)               # 5
        self.conv3 = nn.Conv2d(32, 16, 3, padding=0)  # 3
        self.conv4 = nn.Conv2d(16, 10, 3, padding=0)  # 1

        

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.conv4(F.relu(self.conv3(x)))
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net8().to(device), input_size=(1, 28, 28))

# rewrite. Without padding. only 4 conv layers. num ch capped to 32.
# 10.8k params
#Acc: 98.43 in 5

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
         MaxPool2d-2           [-1, 16, 13, 13]               0
            Conv2d-3           [-1, 32, 11, 11]           4,640
         MaxPool2d-4             [-1, 32, 5, 5]               0
            Conv2d-5             [-1, 16, 3, 3]           4,624
            Conv2d-6             [-1, 10, 1, 1]           1,450
Total params: 10,874
Trainable params: 10,874
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.14
Params size (MB): 0.04
Estimated Total Size (MB): 0.18
----------------------------------------------------------------


<ipython-input-9-8ad78ad7ac8c>:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [10]:
class Net9(nn.Module):
    def __init__(self):
        super(Net9, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   # 28
        self.pool1 = nn.MaxPool2d(2, 2)               # 14
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 14
        self.pool2 = nn.MaxPool2d(2, 2)               # 7
        self.conv3 = nn.Conv2d(32, 16, 3, padding=0)  # 5
        self.conv4 = nn.Conv2d(16, 16, 3, padding=0)  # 3
        self.conv5 = nn.Conv2d(16, 10, 3, padding=0)  # 1

        

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.conv5(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net9().to(device), input_size=(1, 28, 28))

# re-added padding, params increased. added a conv layer
# 13k params
#Acc: 98.55 in 5

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
         MaxPool2d-2           [-1, 16, 14, 14]               0
            Conv2d-3           [-1, 32, 14, 14]           4,640
         MaxPool2d-4             [-1, 32, 7, 7]               0
            Conv2d-5             [-1, 16, 5, 5]           4,624
            Conv2d-6             [-1, 16, 3, 3]           2,320
            Conv2d-7             [-1, 10, 1, 1]           1,450
Total params: 13,194
Trainable params: 13,194
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.18
Params size (MB): 0.05
Estimated Total Size (MB): 0.24
----------------------------------------------------------------


<ipython-input-10-7a22a28ccb68>:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [11]:
class Net10(nn.Module):
    def __init__(self):
        super(Net10, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   # 28
        self.pool1 = nn.MaxPool2d(2, 2)               # 14
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 14
        self.pool2 = nn.MaxPool2d(2, 2)               # 7
        self.conv3 = nn.Conv2d(32, 16, 3, padding=0)  # 5
        self.conv4 = nn.Conv2d(16, 8, 3, padding=0)  # 3
        self.conv5 = nn.Conv2d(8, 10, 3, padding=0)  # 1

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.conv5(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net10().to(device), input_size=(1, 28, 28))

# reduced conv4 num ch
# 11k params
#Acc: 98.23 in 5

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
         MaxPool2d-2           [-1, 16, 14, 14]               0
            Conv2d-3           [-1, 32, 14, 14]           4,640
         MaxPool2d-4             [-1, 32, 7, 7]               0
            Conv2d-5             [-1, 16, 5, 5]           4,624
            Conv2d-6              [-1, 8, 3, 3]           1,160
            Conv2d-7             [-1, 10, 1, 1]             730
Total params: 11,314
Trainable params: 11,314
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.18
Params size (MB): 0.04
Estimated Total Size (MB): 0.23
----------------------------------------------------------------


<ipython-input-11-d3031bf56399>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [12]:
class Net11(nn.Module):
    def __init__(self):
        super(Net11, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   # 28
        self.bn1 = nn.BatchNorm2d(16)

        self.pool1 = nn.MaxPool2d(2, 2)               # 14

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 14
        self.bn2 = nn.BatchNorm2d(32)
        
        self.pool2 = nn.MaxPool2d(2, 2)               # 7
        
        self.conv3 = nn.Conv2d(32, 16, 3, padding=0)  # 5
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 8, 3, padding=0)  # 3
        self.bn4 = nn.BatchNorm2d(8)
        self.conv5 = nn.Conv2d(8, 10, 3, padding=0)  # 1

    def forward(self, x):
        x = self.pool1(self.bn1(F.relu(self.conv1(x))))
        x = self.pool2(self.bn2(F.relu(self.conv2(x))))
        x = self.conv5(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net11().to(device), input_size=(1, 28, 28))

# added batch norms
# 11k params
#Acc: 98.76 in 5


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           4,640
       BatchNorm2d-5           [-1, 32, 14, 14]              64
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 16, 5, 5]           4,624
       BatchNorm2d-8             [-1, 16, 5, 5]              32
            Conv2d-9              [-1, 8, 3, 3]           1,160
      BatchNorm2d-10              [-1, 8, 3, 3]              16
           Conv2d-11             [-1, 10, 1, 1]             730
Total params: 11,458
Trainable params: 11,458
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

<ipython-input-12-316ec9c74abd>:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [13]:
class Net12(nn.Module):
    def __init__(self):
        super(Net12, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   # 28
        self.bn1 = nn.BatchNorm2d(16)

        self.pool1 = nn.MaxPool2d(2, 2)               # 14
        self.do1 = nn.Dropout(0.1)
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 14
        self.bn2 = nn.BatchNorm2d(32)
        
        self.pool2 = nn.MaxPool2d(2, 2)               # 7
        self.do2 = nn.Dropout(0.1)
        
        self.conv3 = nn.Conv2d(32, 16, 3, padding=0)  # 5
        self.bn3 = nn.BatchNorm2d(16)
        self.do3 = nn.Dropout(0.1)

        self.conv4 = nn.Conv2d(16, 8, 3, padding=0)  # 3
        self.bn4 = nn.BatchNorm2d(8)
        self.do4 = nn.Dropout(0.1)

        self.conv5 = nn.Conv2d(8, 10, 3, padding=0)  # 1

    def forward(self, x):
        x = self.pool1(self.bn1(F.relu(self.conv1(x))))
        x = self.do1(x)

        x = self.pool2(self.bn2(F.relu(self.conv2(x))))
        x = self.do2(x)

        x = self.bn3(F.relu(self.conv3(x)))
        x = self.do3(x)
        
        x = self.bn4(F.relu(self.conv4(x)))
        x = self.do4(x)

        x = self.conv5(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

summary(Net12().to(device), input_size=(1, 28, 28))

# added dropouts
# 11.5k params
# Acc: 99.0 in 5, 99.23 in 20

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
           Dropout-4           [-1, 16, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           4,640
       BatchNorm2d-6           [-1, 32, 14, 14]              64
         MaxPool2d-7             [-1, 32, 7, 7]               0
           Dropout-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 16, 5, 5]           4,624
      BatchNorm2d-10             [-1, 16, 5, 5]              32
          Dropout-11             [-1, 16, 5, 5]               0
           Conv2d-12              [-1, 8, 3, 3]           1,160
      BatchNorm2d-13              [-1, 8, 3, 3]              16
          Dropout-14              [-1, 

<ipython-input-13-c71d6df545ca>:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [21]:
class Net13(nn.Module):
    def __init__(self):
        super(Net13, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   # 28
        self.bn1 = nn.BatchNorm2d(16)

        self.pool1 = nn.MaxPool2d(2, 2)               # 14
        self.do1 = nn.Dropout(0.1)
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 14
        self.bn2 = nn.BatchNorm2d(32)
        
        self.pool2 = nn.MaxPool2d(2, 2)               # 7
        self.do2 = nn.Dropout(0.1)
        
        self.conv3 = nn.Conv2d(32, 16, 3, padding=0)  # 5
        self.bn3 = nn.BatchNorm2d(16)
        self.do3 = nn.Dropout(0.1)

        self.fc1 = nn.Linear(5 * 5 * 16, 10)          # fc

    def forward(self, x):
        x = self.do1(self.pool1(self.bn1(F.relu(self.conv1(x)))))

        x = self.do2(self.pool2(self.bn2(F.relu(self.conv2(x)))))

        x = self.do3(self.bn3(F.relu(self.conv3(x))))
        
        x = x.view(-1, 5 * 5 * 16)
        x = self.fc1(x)

        return F.log_softmax(x, dim=0)

summary(Net13().to(device), input_size=(1, 28, 28))

# replace conv4, conv5 with fc
# 13.5k params
# Acc: 99.04 in 5

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
           Dropout-4           [-1, 16, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           4,640
       BatchNorm2d-6           [-1, 32, 14, 14]              64
         MaxPool2d-7             [-1, 32, 7, 7]               0
           Dropout-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 16, 5, 5]           4,624
      BatchNorm2d-10             [-1, 16, 5, 5]              32
          Dropout-11             [-1, 16, 5, 5]               0
           Linear-12                   [-1, 10]           4,010
Total params: 13,562
Trainable params: 13,562
Non-trainable params: 0
---------------------------------

In [25]:
class Net14(nn.Module):
    def __init__(self):
        super(Net14, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   # 28
        self.bn1 = nn.BatchNorm2d(16)

        self.pool1 = nn.MaxPool2d(2, 2)               # 14
        self.do1 = nn.Dropout(0.1)
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 14
        self.bn2 = nn.BatchNorm2d(32)
        
        self.pool2 = nn.MaxPool2d(2, 2)               # 7
        self.do2 = nn.Dropout(0.1)
        
        self.conv3 = nn.Conv2d(32, 16, 3, padding=0)  # 5
        self.bn3 = nn.BatchNorm2d(16)
        self.do3 = nn.Dropout(0.1)

        self.conv4 = nn.Conv2d(16, 10, 3, padding=0)  # 3
        self.gap = nn.AvgPool2d(3)

    def forward(self, x):
        x = self.do1(self.pool1(self.bn1(F.relu(self.conv1(x)))))

        x = self.do2(self.pool2(self.bn2(F.relu(self.conv2(x)))))

        x = self.do3(self.bn3(F.relu(self.conv3(x))))
        
        x = self.conv4(x)
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=0)

summary(Net14().to(device), input_size=(1, 28, 28))

# Replace fc by conv4 (16, 10, 3) + gap(3)
# 11k
# Acc: 98.63 in 5, 98.9 in 20

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
           Dropout-4           [-1, 16, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           4,640
       BatchNorm2d-6           [-1, 32, 14, 14]              64
         MaxPool2d-7             [-1, 32, 7, 7]               0
           Dropout-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 16, 5, 5]           4,624
      BatchNorm2d-10             [-1, 16, 5, 5]              32
          Dropout-11             [-1, 16, 5, 5]               0
           Conv2d-12             [-1, 10, 3, 3]           1,450
        AvgPool2d-13             [-1, 10, 1, 1]               0
Total params: 11,002
Trainable params: 

In [18]:
def run(model, num_epochs=5):
  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

  for epoch in range(1, num_epochs):
      train(model, device, train_loader, optimizer, epoch)
      test(model, device, test_loader)

def run2(model, num_epochs=5):
  optimizer = optim.Adam(model.parameters(), lr=0.01)

  for epoch in range(1, num_epochs):
      train(model, device, train_loader, optimizer, epoch)
      test(model, device, test_loader)

In [28]:
run2(Net14().to(device))

loss=1.3940106630325317 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 45.99it/s]



Test set: Average loss: 2.0532, Accuracy: 9788/10000 (98%)



loss=1.4405502080917358 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 46.70it/s]



Test set: Average loss: 2.0263, Accuracy: 9818/10000 (98%)



loss=1.343474268913269 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 45.85it/s]



Test set: Average loss: 2.0101, Accuracy: 9840/10000 (98%)



loss=1.4018528461456299 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 46.67it/s]



Test set: Average loss: 2.0040, Accuracy: 9834/10000 (98%)



In [29]:
run2(Net14().to(device), num_epochs=20)

loss=1.375826120376587 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 46.53it/s]



Test set: Average loss: 2.0469, Accuracy: 9801/10000 (98%)



loss=1.4711716175079346 batch_id=937: 100%|██████████| 938/938 [00:19<00:00, 47.07it/s]



Test set: Average loss: 2.0136, Accuracy: 9855/10000 (99%)



loss=1.372450351715088 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 45.55it/s]


...


Test set: Average loss: 1.9831, Accuracy: 9894/10000 (99%)



loss=1.453381061553955 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 45.07it/s]



Test set: Average loss: 1.9756, Accuracy: 9896/10000 (99%)



loss=1.2905714511871338 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 44.90it/s]



Test set: Average loss: 1.9689, Accuracy: 9890/10000 (99%)

